# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *A non-local algorithm for image denoising* autorstwa Antoni Buades, Bartomeu Coll, i Jean Michel Morel na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation}

gdzie:
- \begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in  V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation},
- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując. Jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
from time import time

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

        
mat = loadmat('MR_data.mat')

#### Filtr Gaussa

In [ ]:
def gaussian (context, mask):
    return np.sum(context * mask)
    

def classic (I, sigma, size):
    mask = np.zeros((size, size))
    for i in range (size):
        for j in range (size):
            x, y = size//2 - i, size//2 - j
            mask[i, j] = np.exp(-(x**2 + y**2)/(2*sigma**2))
    mask = mask / np.sum(mask)
    
   
    Iconv = I.copy()
    delta = size//2
    for i in range (delta, Iconv.shape[0] - delta):
        for j in range (delta, Iconv.shape[0] - delta):
            context = Iconv[i-delta : i+delta+1, j-delta : j+delta+1]
            Iconv[i, j] = gaussian(context, mask)
    return Iconv

#### Filtr Bilateralny

In [ ]:
def bilateral_fun (context, mask, sigma_r):
    size = context.shape[0]
    p = context[size//2, size//2]
    gamma = lambda y: np.exp((y-p)**2 / (-2 *  sigma_r**2))
    mask2 = gamma(context.copy())
    return np.sum(context * mask * mask2) / np.sum(mask2*mask)
    

def bilateral (I, sigma, size, sigma_r):
    mask = np.zeros((size, size))
    for i in range (size):
        for j in range (size):
            x, y = size//2 - i, size//2 - j
            mask[i, j] = np.exp(-(x**2 + y**2)/(2*sigma**2))
    
    
    Iconv = I.copy()
    delta = size//2
    for i in range (delta, Iconv.shape[0] - delta):
        for j in range (delta, Iconv.shape[0] - delta):
            context = Iconv[i-delta : i+delta+1, j-delta : j+delta+1]
            Iconv[i, j] = bilateral_fun(context, mask, sigma_r)
    return Iconv

#### Filtr NLM

In [ ]:
def nlm_fun(context, mask, size_N, alfa, sigma_r):
    w_sum = 0
    val = 0
    delta = size_N//2
    size = context.shape[0]
    context_p = context[size//2-delta : size//2+delta+1, size//2-delta : size//2+delta+1]
    gamma = lambda y: np.sqrt(np.sum((mask*y - mask*context_p)**2))
    p = context[size//2, size//2]
    for i in range (delta, context.shape[0] - delta):
        for j in range (delta, context.shape[0] - delta):
            context_N = context[i-delta : i+delta+1, j-delta : j+delta+1]
            w = np.exp((-1) * gamma(context_N) / (alfa * sigma_r**2))
            w_sum += w
            val += p*w
    return val / w_sum


def nlm (I, size_V, size_N, alfa, sigma, sigma_r):
    mask = np.zeros((size_N, size_N))
    for i in range (size_N):
        for j in range (size_N):
            x, y = size_N//2 - i, size_N//2 - j
            mask[i, j] = np.exp(-(x**2 + y**2)/(2*sigma**2))
    mask = mask / np.sum(mask)
    
    
    Iconv = I.copy()    
    delta = size_V//2 + size_N//2
    for i in range (delta, Iconv.shape[0] - delta):
        for j in range (delta, Iconv.shape[0] - delta):
            context = Iconv[i-delta : i+delta+1, j-delta : j+delta+1]
            Iconv[i, j] = nlm_fun(context, mask, size_N, alfa, sigma_r)
    return Iconv

#### Porównanie efektów

In [ ]:
I = mat['I_noisefree']

times = []
labels = ["gaussian", "bilateral", "nlm"]

start = time()
I_gaussian = classic(I, 0.7, 9)
end = time()
times.append(end-start)

start = time()
I_bilateral = bilateral(I, 2, 5, 16)
end = time()
times.append(end-start)

start = time()
I_nlm = nlm(I, 7, 3, 8, 0.7, 40)
end = time()
times.append(end-start)


fig, axs = plt.subplots(3, 2)
fig.set_size_inches(20, 30)

axs[0, 0].imshow(I, 'gray', vmin=0, vmax=256)
axs[0, 0].axis('off')
axs[0, 0].set_title("oryginał")
axs[0, 1].imshow(I_gaussian, 'gray', vmin=0, vmax=256)
axs[0, 1].axis('off')
axs[0, 1].set_title("Gaussian")

axs[1, 0].imshow(I_bilateral, 'gray', vmin=0, vmax=256)
axs[1, 0].axis('off')
axs[1, 0].set_title("Bilateralny")
axs[1, 1].imshow(I_nlm, 'gray', vmin=0, vmax=256)
axs[1, 1].axis('off')
axs[1, 1].set_title("NLM")

axs[2, 0].bar(labels, times)
axs[2, 0].grid()
axs[2, 0].set_title("Czasy")
axs[2, 1].axis('off')

In [ ]:
I = mat['I_noisy1']

times = []
labels = ["gaussian", "bilateral", "nlm"]

start = time()
I_gaussian = classic(I, 0.7, 9)
end = time()
times.append(end-start)

start = time()
I_bilateral = bilateral(I, 2, 5, 16)
end = time()
times.append(end-start)

start = time()
I_nlm = nlm(I, 7, 3, 8, 0.7, 40)
end = time()
times.append(end-start)


fig, axs = plt.subplots(3, 2)
fig.set_size_inches(20, 30)

axs[0, 0].imshow(I, 'gray', vmin=0, vmax=256)
axs[0, 0].axis('off')
axs[0, 0].set_title("oryginał")
axs[0, 1].imshow(I_gaussian, 'gray', vmin=0, vmax=256)
axs[0, 1].axis('off')
axs[0, 1].set_title("Gaussian")

axs[1, 0].imshow(I_bilateral, 'gray', vmin=0, vmax=256)
axs[1, 0].axis('off')
axs[1, 0].set_title("Bilateralny")
axs[1, 1].imshow(I_nlm, 'gray', vmin=0, vmax=256)
axs[1, 1].axis('off')
axs[1, 1].set_title("NLM")

axs[2, 0].bar(labels, times)
axs[2, 0].grid()
axs[2, 0].set_title("Czasy")
axs[2, 1].axis('off')

In [ ]:
I = mat['I_noisy2']

times = []
labels = ["gaussian", "bilateral", "nlm"]

start = time()
I_gaussian = classic(I, 0.7, 9)
end = time()
times.append(end-start)

start = time()
I_bilateral = bilateral(I, 2, 5, 16)
end = time()
times.append(end-start)

start = time()
I_nlm = nlm(I, 7, 3, 8, 0.7, 40)
end = time()
times.append(end-start)


fig, axs = plt.subplots(3, 2)
fig.set_size_inches(20, 30)

axs[0, 0].imshow(I, 'gray', vmin=0, vmax=256)
axs[0, 0].axis('off')
axs[0, 0].set_title("oryginał")
axs[0, 1].imshow(I_gaussian, 'gray', vmin=0, vmax=256)
axs[0, 1].axis('off')
axs[0, 1].set_title("Gaussian")

axs[1, 0].imshow(I_bilateral, 'gray', vmin=0, vmax=256)
axs[1, 0].axis('off')
axs[1, 0].set_title("Bilateralny")
axs[1, 1].imshow(I_nlm, 'gray', vmin=0, vmax=256)
axs[1, 1].axis('off')
axs[1, 1].set_title("NLM")

axs[2, 0].bar(labels, times)
axs[2, 0].grid()
axs[2, 0].set_title("Czasy")
axs[2, 1].axis('off')

In [ ]:
I = mat['I_noisy3']

times = []
labels = ["gaussian", "bilateral", "nlm"]

start = time()
I_gaussian = classic(I, 0.7, 9)
end = time()
times.append(end-start)

start = time()
I_bilateral = bilateral(I, 2, 5, 16)
end = time()
times.append(end-start)

start = time()
I_nlm = nlm(I, 7, 3, 8, 0.7, 40)
end = time()
times.append(end-start)


fig, axs = plt.subplots(3, 2)
fig.set_size_inches(20, 30)

axs[0, 0].imshow(I, 'gray', vmin=0, vmax=256)
axs[0, 0].axis('off')
axs[0, 0].set_title("oryginał")
axs[0, 1].imshow(I_gaussian, 'gray', vmin=0, vmax=256)
axs[0, 1].axis('off')
axs[0, 1].set_title("Gaussian")

axs[1, 0].imshow(I_bilateral, 'gray', vmin=0, vmax=256)
axs[1, 0].axis('off')
axs[1, 0].set_title("Bilateralny")
axs[1, 1].imshow(I_nlm, 'gray', vmin=0, vmax=256)
axs[1, 1].axis('off')
axs[1, 1].set_title("NLM")

axs[2, 0].bar(labels, times)
axs[2, 0].grid()
axs[2, 0].set_title("Czasy")
axs[2, 1].axis('off')

In [ ]:
I = mat['I_noisy4']

times = []
labels = ["gaussian", "bilateral", "nlm"]

start = time()
I_gaussian = classic(I, 0.7, 9)
end = time()
times.append(end-start)

start = time()
I_bilateral = bilateral(I, 2, 5, 16)
end = time()
times.append(end-start)

start = time()
I_nlm = nlm(I, 7, 3, 8, 0.7, 40)
end = time()
times.append(end-start)


fig, axs = plt.subplots(3, 2)
fig.set_size_inches(20, 30)

axs[0, 0].imshow(I, 'gray', vmin=0, vmax=256)
axs[0, 0].axis('off')
axs[0, 0].set_title("oryginał")
axs[0, 1].imshow(I_gaussian, 'gray', vmin=0, vmax=256)
axs[0, 1].axis('off')
axs[0, 1].set_title("Gaussian")

axs[1, 0].imshow(I_bilateral, 'gray', vmin=0, vmax=256)
axs[1, 0].axis('off')
axs[1, 0].set_title("Bilateralny")
axs[1, 1].imshow(I_nlm, 'gray', vmin=0, vmax=256)
axs[1, 1].axis('off')
axs[1, 1].set_title("NLM")

axs[2, 0].bar(labels, times)
axs[2, 0].grid()
axs[2, 0].set_title("Czasy")
axs[2, 1].axis('off')

Dla NLM wyniki wydają się najlepsze. Metoda jest najbardziej rozbudowana i ma na nią wpływ najwięcej danych. Pozwala poza analizowaniem kontekstu jak w bilateralnym na analizę kontekstu punktów z kontekstu. Duże znaczenie ma odpowiedni dobór parametrów. Filtr Bilteralny działa dużo szybciej a też bardzo dobrze sobie radzi.

Czas działania NLM jest dużo dłuższy w porównaniu do dwóch pozostałych metod. Nie jest to zaskakujące, w filtru Gaussowskim wykonujemy jedną operację na wyciętym oknie, w filtru Bilateralnym kilka operacji a w NLM wycinamy kolejne podokna i na nich wykonujemy operacje. Widać że złożoność jest większa dla NLM, a dla Gaussa i Bilateralnego taka sama.